In [1]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# No ReVIN no series decomposition

In [2]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "PatchTST"
dataset = 'DE_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max_0_1_relu"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [3]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}" \
              --revin 0 \
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=0, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0

In [4]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu_no_revin.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu_no_revin.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0228  0.1509  0.0993  0.5327
              2         24        0.0232  0.1522  0.0993  0.5374
              1         96        0.0423  0.2058  0.1442  0.7287
              2         96        0.0382  0.1954  0.1364  0.6921
              1         168       0.0446  0.2112  0.1488  0.7481
              2         168       0.0429  0.2072  0.1472  0.7342
RMSE          1         24        0.0247  0.1573  0.1037  0.5555
              2         24        0.0276  0.1662  0.1114  0.5871
              1         96        0.0418  0.2044  0.1435  0.7237
              2         96        0.0390  0.1976  0.1391  0.6997
              1         168       0.0409  0.2023  0.1403  0.7166
              2         168       0.0426  0.2065  0.1467  0.7316
MAE           1         24        0.0228  0.1511  0.0952  0.5337
              2         24        0.0235  0.1532  0.0949  0.5412
              1         96        0.0393  0.1982  0.1341  0.7019
              2         96        0.0423  0.2057  0.1356  0.7286
              1         168       0.0512  0.2262  0.1500  0.8015
              2         168       0.0506  0.2249  0.1500  0.7967

In [5]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        17937842.0  4235.3091  2700.5715  0.2106
              2         24        18291816.0  4276.8931  2723.5989  0.2127
              1         96        37627388.0  6134.1167  4097.5498  0.3055
              2         96        34112108.0  5840.5571  3830.1448  0.2909
              1         168       40797516.0  6387.2935  4201.8149  0.3182
              2         168       40016304.0  6325.8442  4216.5996  0.3152
RMSE          1         24        20196358.0  4494.0356  2889.8323  0.2235
              2         24        23053352.0  4801.3906  3150.2073  0.2387
              1         96        37400496.0  6115.5947  4083.6396  0.3046
              2         96        34960184.0  5912.7139  3935.2871  0.2945
              1         168       35232612.0  5935.7065  3864.7725  0.2957
              2         168       39467032.0  6282.2793  4191.4492  0.3130
MAE           1         24        17627044.0  4198.4575  2560.0320  0.2088
              2         24        18100182.0  4254.4307  2555.0552  0.2115
              1         96        32837766.0  5730.4248  3682.6191  0.2854
              2         96        33165298.0  5758.9321  3640.1892  0.2868
              1         168       41286724.0  6425.4746  4055.5706  0.3201
              2         168       40031320.0  6327.0308  4023.6497  0.3152

In [6]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0232  0.1522  0.0951  0.5375
         MSE            0.0230  0.1515  0.0993  0.5351
         RMSE           0.0262  0.1618  0.1075  0.5713
96       MAE            0.0408  0.2020  0.1349  0.7152
         MSE            0.0403  0.2006  0.1403  0.7104
         RMSE           0.0404  0.2010  0.1413  0.7117
168      MAE            0.0509  0.2256  0.1500  0.7991
         MSE            0.0438  0.2092  0.1480  0.7412
         RMSE           0.0418  0.2044  0.1435  0.7241

In [7]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            17863613.0  4226.4441  2557.5436  0.2101
         MSE            18114829.0  4256.1011  2712.0852  0.2116
         RMSE           21624855.0  4647.7131  3020.0198  0.2311
96       MAE            33001532.0  5744.6785  3661.4042  0.2861
         MSE            35869748.0  5987.3369  3963.8473  0.2982
         RMSE           36180340.0  6014.1543  4009.4634  0.2995
168      MAE            40659022.0  6376.2527  4039.6101  0.3177
         MSE            40406910.0  6356.5688  4209.2073  0.3167
         RMSE           37349822.0  6108.9929  4028.1108  0.3044

In [ ]:
# Rename folders
new_path_name = 'minmax_0_1_relu_unscaled_no_revin'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)

# TS decomposition + ReVIN

In [11]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}" \
              --decomposition 1 
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=1, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0

In [12]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu_decomposition.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu_decomposition.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0238  0.1542  0.0996  0.5445
              2         24        0.0227  0.1507  0.0971  0.5323
              1         96        0.0368  0.1917  0.1393  0.6789
              2         96        0.0366  0.1914  0.1388  0.6777
              1         168       0.0390  0.1974  0.1455  0.6994
              2         168       0.0391  0.1978  0.1451  0.7008
RMSE          1         24        0.0237  0.1540  0.0995  0.5440
              2         24        0.0233  0.1527  0.0980  0.5394
              1         96        0.0366  0.1913  0.1384  0.6775
              2         96        0.0365  0.1910  0.1380  0.6764
              1         168       0.0389  0.1971  0.1448  0.6984
              2         168       0.0390  0.1975  0.1445  0.6997
MAE           1         24        0.0233  0.1527  0.0958  0.5393
              2         24        0.0231  0.1520  0.0949  0.5368
              1         96        0.0384  0.1960  0.1306  0.6942
              2         96        0.0379  0.1947  0.1312  0.6896
              1         168       0.0389  0.1972  0.1351  0.6987
              2         168       0.0391  0.1979  0.1353  0.7010

In [13]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        18796964.0  4335.5464  2721.7759  0.2156
              2         24        18245456.0  4271.4702  2664.4333  0.2124
              1         96        32576136.0  5707.5508  3937.2654  0.2842
              2         96        32513784.0  5702.0859  3915.3999  0.2840
              1         168       35581968.0  5965.0625  4137.5913  0.2972
              2         168       35745328.0  5978.7397  4123.7764  0.2979
RMSE          1         24        19014310.0  4360.5400  2722.3164  0.2168
              2         24        18353394.0  4284.0859  2661.8276  0.2130
              1         96        32360570.0  5688.6353  3905.5920  0.2833
              2         96        32326948.0  5685.6792  3887.6257  0.2831
              1         168       35409284.0  5950.5698  4112.2773  0.2965
              2         168       35583288.0  5965.1729  4101.0981  0.2972
MAE           1         24        18010934.0  4243.9292  2587.8503  0.2110
              2         24        17940912.0  4235.6714  2567.2988  0.2106
              1         96        33274860.0  5768.4365  3605.2803  0.2873
              2         96        32594430.0  5709.1533  3625.5750  0.2843
              1         168       34130992.0  5842.1733  3749.2737  0.2911
              2         168       34361140.0  5861.8374  3752.2725  0.2921

In [14]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0232  0.1523  0.0953  0.5380
         MSE            0.0232  0.1525  0.0984  0.5384
         RMSE           0.0235  0.1534  0.0988  0.5417
96       MAE            0.0382  0.1954  0.1309  0.6919
         MSE            0.0367  0.1915  0.1390  0.6783
         RMSE           0.0365  0.1912  0.1382  0.6770
168      MAE            0.0390  0.1975  0.1352  0.6998
         MSE            0.0391  0.1976  0.1453  0.7001
         RMSE           0.0389  0.1973  0.1447  0.6991

In [15]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            17975923.0  4239.8003  2577.5746  0.2108
         MSE            18521210.0  4303.5083  2693.1046  0.2140
         RMSE           18683852.0  4322.3130  2692.0720  0.2149
96       MAE            32934645.0  5738.7949  3615.4276  0.2858
         MSE            32544960.0  5704.8184  3926.3326  0.2841
         RMSE           32343759.0  5687.1572  3896.6089  0.2832
168      MAE            34246066.0  5852.0054  3750.7731  0.2916
         MSE            35663648.0  5971.9011  4130.6838  0.2975
         RMSE           35496286.0  5957.8713  4106.6877  0.2968

In [16]:
# Rename folders
new_path_name = 'minmax_0_1_relu_unscaled_decomposition'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)

# TS decomposition and no ReVIN

In [17]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}" \
              --revin 0 \
              --decomposition 1
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=0, affine=0, subtract_last=0, decomposition=1, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0

In [23]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu_no_revin_decomposition.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu_no_revin_decomposition.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0254  0.1593  0.1056  0.5624
              2         24        0.0241  0.1551  0.1024  0.5477
              1         96        0.0420  0.2050  0.1436  0.7258
              2         96        0.0425  0.2062  0.1425  0.7303
              1         168       0.0449  0.2119  0.1500  0.7508
              2         168       0.0481  0.2193  0.1509  0.7768
RMSE          1         24        0.0239  0.1547  0.1037  0.5462
              2         24        0.0240  0.1550  0.1044  0.5476
              1         96        0.0476  0.2182  0.1525  0.7726
              2         96        0.0458  0.2140  0.1418  0.7578
              1         168       0.0443  0.2106  0.1473  0.7460
              2         168       0.0447  0.2115  0.1491  0.7491
MAE           1         24        0.0270  0.1644  0.1032  0.5805
              2         24        0.0249  0.1577  0.0996  0.5568
              1         96        0.0496  0.2226  0.1465  0.7884
              2         96        0.0496  0.2226  0.1467  0.7883
              1         168       0.0603  0.2456  0.1604  0.8701
              2         168       0.0545  0.2335  0.1560  0.8273

In [24]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        19506484.0  4416.6147  2876.1069  0.2196
              2         24        19008062.0  4359.8237  2790.5967  0.2168
              1         96        36130940.0  6010.9019  4012.9888  0.2993
              2         96        36122992.0  6010.2407  3954.7468  0.2993
              1         168       39282812.0  6267.6001  4202.2949  0.3123
              2         168       40928480.0  6397.5371  4202.4697  0.3188
RMSE          1         24        19043724.0  4363.9116  2849.0442  0.2170
              2         24        19401690.0  4404.7349  2870.5693  0.2190
              1         96        38907712.0  6237.6045  4236.4297  0.3106
              2         96        37945684.0  6160.0068  3876.0593  0.3068
              1         168       38489120.0  6203.9600  4120.3130  0.3091
              2         168       39429468.0  6279.2886  4190.6279  0.3129
MAE           1         24        19359624.0  4399.9570  2728.8313  0.2188
              2         24        18901044.0  4347.5332  2686.6917  0.2162
              1         96        38852912.0  6233.2104  3966.3728  0.3104
              2         96        39954652.0  6320.9692  3986.6050  0.3148
              1         168       43195992.0  6572.3657  4203.3413  0.3275
              2         168       45199216.0  6723.0361  4282.5435  0.3350

In [25]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0259  0.1610  0.1014  0.5687
         MSE            0.0247  0.1572  0.1040  0.5551
         RMSE           0.0240  0.1549  0.1040  0.5469
96       MAE            0.0496  0.2226  0.1466  0.7883
         MSE            0.0423  0.2056  0.1431  0.7281
         RMSE           0.0467  0.2161  0.1472  0.7652
168      MAE            0.0574  0.2396  0.1582  0.8487
         MSE            0.0465  0.2156  0.1504  0.7638
         RMSE           0.0445  0.2110  0.1482  0.7476

In [26]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            19130334.0  4373.7451  2707.7615  0.2175
         MSE            19257273.0  4388.2192  2833.3518  0.2182
         RMSE           19222707.0  4384.3232  2859.8068  0.2180
96       MAE            39403782.0  6277.0898  3976.4889  0.3126
         MSE            36126966.0  6010.5713  3983.8678  0.2993
         RMSE           38426698.0  6198.8057  4056.2445  0.3087
168      MAE            44197604.0  6647.7009  4242.9424  0.3312
         MSE            40105646.0  6332.5686  4202.3823  0.3155
         RMSE           38959294.0  6241.6243  4155.4705  0.3110

In [ ]:
# Rename folders
new_path_name = 'minmax_0_1_relu_unscaled_no_revin_decomposition'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)